In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importando Bibliotecas

In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.tree import DecisionTreeClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import unicodedata
import nltk


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Carregando dataset

In [29]:
path_dataset = '/content/drive/Shareddrives/Aprendizado por reforço/NLP/Dataset/dataset_projeto_final.csv'
df = pd.read_csv(path_dataset)

# Pré-processamento

In [30]:
top = stopwords.words('portuguese')
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (top)]))
symbols = "!\"#$%&()*+-,./:´~;<=>?[\]^`{|}~\n..."
df["text"] = df["text"].str.translate({ord(symbol):"" for symbol in symbols})
df['text'] = df['text'].str.lower()
df['text'] = df['text'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [31]:
# As classes 'Notificação de inscrição' e 'Inscrição indevida' são parecidas, então optamos por juntá-las em uma só
df['label'] = df['label'].replace(['Notificação de inscrição'], 'Inscrição indevida')

In [32]:
df["label"].value_counts()

Dano moral                                   9280
Inexistência de relação jurídica/débito      8843
Jurisprudência                               4863
Tutela antecipada                            4610
Inscrição indevida                           3926
Inversão do ônus da prova                    3649
Fraude                                       3444
Qualificação                                 2832
Justiça gratuita                             2718
Falha prestação/responsabilidade objetiva    2572
Relação de consumo                           2143
Produtos bancários                           1900
Dano material                                1707
Restituição em dobro                         1623
Responsabilidade civil                       1549
Canais Internos                              1468
Valor da causa                               1330
Produção de provas                           1132
Audiência conciliação                         843
Honorários/custas                             829


# Separando o dataset em treino e teste

In [33]:
train, test = train_test_split(df, test_size=0.2, random_state=17, stratify = df['label'])

In [34]:
train_x, y_train= train['text'], train['label']
test_x, y_test = test['text'], test['label']

#TFIDF

In [35]:
tfidf = TfidfVectorizer(stop_words = top)
train_x_vector = tfidf.fit_transform(train_x)
train_x_vector

<51338x55994 sparse matrix of type '<class 'numpy.float64'>'
	with 1435776 stored elements in Compressed Sparse Row format>

In [36]:
test_x_vector = tfidf.transform(test_x)


In [37]:
y_test.value_counts()

Dano moral                                   1856
Inexistência de relação jurídica/débito      1769
Jurisprudência                                973
Tutela antecipada                             922
Inscrição indevida                            785
Inversão do ônus da prova                     730
Fraude                                        689
Qualificação                                  566
Justiça gratuita                              544
Falha prestação/responsabilidade objetiva     514
Relação de consumo                            429
Produtos bancários                            380
Dano material                                 341
Restituição em dobro                          325
Responsabilidade civil                        310
Canais Internos                               294
Valor da causa                                266
Produção de provas                            226
Audiência conciliação                         169
Honorários/custas                             166


# Usando árvore de decisão para classificação

In [38]:
# Arvore de decisão
random_forest = RandomForestClassifier()
random_forest.fit(train_x_vector, y_train)

RandomForestClassifier()

In [39]:
y_pred = random_forest.predict(test_x_vector)

In [40]:
print(classification_report(y_test, y_pred))

                                           precision    recall  f1-score   support

                    Audiência conciliação       0.90      0.95      0.92       169
                        Benefeciário INSS       0.83      0.28      0.42        18
                          Canais Internos       0.70      0.61      0.65       294
                      Citação/contestação       0.91      0.89      0.90       148
                            Dano material       0.82      0.34      0.48       341
                               Dano moral       0.82      0.91      0.86      1856
                         Desvio produtivo       0.85      0.78      0.81       156
Falha prestação/responsabilidade objetiva       0.78      0.66      0.71       514
                    Financiamento veículo       0.92      0.50      0.65       112
                                   Fraude       0.85      0.63      0.72       689
                        Honorários/custas       0.93      0.94      0.94       166
  I